>> # CUSTOMER CHURN

# Business Understanding

# Data Understanding

In [1]:
#for data manipulation and data reading 
import pandas as pd
#for mathematical computation
import numpy as np
#for visualization 
import matplotlib.pyplot as plt
#for classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import fbeta_score
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score


In [16]:
df = pd.read_csv("customer_churn.csv")
df.head()

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,...,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,...,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,...,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,...,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,375-9999,yes,no,0,299.4,71,50.90,...,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,330-6626,yes,no,0,166.7,113,28.34,...,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [3]:
df.columns

Index(['state', 'account length', 'area code', 'phone number',
       'international plan', 'voice mail plan', 'number vmail messages',
       'total day minutes', 'total day calls', 'total day charge',
       'total eve minutes', 'total eve calls', 'total eve charge',
       'total night minutes', 'total night calls', 'total night charge',
       'total intl minutes', 'total intl calls', 'total intl charge',
       'customer service calls', 'churn'],
      dtype='object')

In [13]:
#function to print out the different data types in our dataframe
def type_of_data(dataframe):
    integer_columns = dataframe.select_dtypes(include="int64")
    float_columns = dataframe.select_dtypes(include="float")
    string_columns = dataframe.select_dtypes(include="object")
    bool_columns = dataframe.select_dtypes(include= "bool")
    
    print(f"There are {len(integer_columns.columns)} numerical columns and the columns are: {list(integer_columns.columns)}")
    print(f"There are {len(float_columns.columns)} decimal columns and the columns are: {list(float_columns.columns)}")
    print(f"There are {len(string_columns.columns)} string columns and the columns are: {list(string_columns.columns)}")
    print(f"There are {len(bool_columns.columns)} boolean columns and the columns are: {list(bool_columns.columns)}")


In [14]:
type_of_data(df)

There are 8 numerical columns and the columns are: ['account length', 'area code', 'number vmail messages', 'total day calls', 'total eve calls', 'total night calls', 'total intl calls', 'customer service calls']
There are 8 decimal columns and the columns are: ['total day minutes', 'total day charge', 'total eve minutes', 'total eve charge', 'total night minutes', 'total night charge', 'total intl minutes', 'total intl charge']
There are 4 string columns and the columns are: ['state', 'phone number', 'international plan', 'voice mail plan']
There are 1 boolean columns and the columns are: ['churn']


### Domain knowledge 

* state: string column about state the customer lives in
* account length: integer  column on number of days the customer has had an account
* area code: integer column about area code of the customer
* phone number: string column containing phone number of the customer
* international plan: string column if the customer has the international plan yes, otherwise no
* voice mail plan:string column if the customer has the voice mail plan yes, otherwise no
* number vmail messages:integer column containing number of voicemails the customer has sent
* total day minutes:float column containing total number of minutes the customer has been in calls during the day
* total day calls:integer column containing total number of calls the user has done during the day
* total day charge:float column containing total amount of money the customer was charged by the Telecom company for calls during the day
* total eve minutes: float column containing total number of minutes the customer has been in calls during the evening
* total eve calls:integer column containing total number of calls the customer has done during the evening
* total eve charge:float column containing total amount of money the customer was charged by the Telecom company for calls during the evening
* total night minutes:float column containg total number of minutes the customer has been in calls during the night
* total night calls:integer column containing total number of calls the customer has done during the night
* total night charge:float column containing total amount of money the customer was charged by the Telecom company for calls during the night
* total intl minutes:float column containing total number of minutes the user has been in international calls
* total intl calls:integer column containing total number of international calls the customer has done
* total intl charge:float column containing total amount of money the customer was charged by the Telecom company for international calls
* customer service calls: integer column containing number of calls the customer has made to customer service
* churn: boolean column containig true if the customer terminated their contract, otherwise false

From the above description it has been noticed that 'phone number' is the only unique identifier and it will be used to check for duplicates and then droped for data privacy and protecting customer data

'churn' will be our target variable because it is the ultimate aim of the company and it is a dependent variable.

# Modeling

# Evaluation